In [ ]:
import intcode

In [ ]:
def load_grid():
    p = intcode.Program(instructions_source='day17.txt')
    p.run()
    outputs = p.output_queue.vals
    
    r = 0
    c = 0
    
    grid = {}
    
    for v in outputs:
        if v == 10:
            r += 1
            c = 0
        else:
            grid[complex(c, r)] = chr(v)
            if chr(v) == '^':
                current_position = complex(c, r)
            c += 1

    current_direction = -1j
    
    return grid, current_position, current_direction

In [ ]:
directions = [1, -1, 1j, -1j]

In [ ]:
grid, current_position, current_direction = load_grid()

In [ ]:
sum(int(p.real * p.imag) for p in grid if grid[p] == '#' and all(grid.get(p + d) == '#' for d in directions))

In [ ]:
def find_next_direction(current_position, current_direction):
    for possible_direction in directions:
        if possible_direction == -current_direction:
            # the way we came from
            continue
        if grid.get(current_position + possible_direction) == '#':
            return possible_direction
    return None

In [ ]:
# Note: these are flipped because increasing y is down.

direction_to_turn = {
    1j: 'R',
    -1j: 'L',
}

In [ ]:
grid, current_position, current_direction = load_grid()

move_order = []

while True:
    next_direction = None
    for possible_direction in directions:
        if possible_direction == -current_direction:
            # the way we came from
            continue
        if grid.get(current_position + possible_direction) == '#':
            next_direction = possible_direction
    
    if next_direction is None:
        # end of the road
        break
        
    move_starts_at = current_position
    
    next_turn = direction_to_turn[next_direction / current_direction]
    current_direction = next_direction
    
    while grid.get(current_position + current_direction) == '#':
        current_position += current_direction
        
    move_ends_at = current_position
    
    length = (move_ends_at - move_starts_at) / current_direction
    length = int(length.real)
        
    move_order.append((next_turn, length))

In [ ]:
def sequence_to_string(sequence):
    cs = []
    for turn, distance in sequence:
        cs.append(f'{turn},{distance}')
    return ','.join(cs)

In [ ]:
start_to_seqs = defaultdict(list)
seq_to_starts = defaultdict(list)

for start in range(len(move_order)):
    for seq_length in range(1, len(move_order)):
        if start + seq_length <= len(move_order):
            seq = tuple(move_order[start:start + seq_length])
            seq_str = sequence_to_string(seq)
            if len(seq_str) <= 20:    
                start_to_seqs[start].append(seq)
                seq_to_starts[seq].append(start)

In [ ]:
def possible_As():
    return start_to_seqs[0]

In [ ]:
def possible_Bs(A):
    B_start = len(A)
    while B_start in seq_to_starts[A]:
        B_start += len(A)
    return start_to_seqs[B_start]

In [ ]:
def possible_Cs(A, B):
    B_start = len(A)
    while B_start in seq_to_starts[A]:
        B_start += len(A)
    C_start = B_start + len(B)
    while C_start in seq_to_starts[A] or C_start in seq_to_starts[B]:
        if C_start in seq_to_starts[A]:
            C_start += len(A)
        else:
            C_start += len(B)
    return start_to_seqs[C_start]

In [ ]:
def covers(A, B, C):
    i = 0
    sub_order = []
    while True:
        if i in seq_to_starts[A]:
            sub_order.append('A')
            i += len(A)
            continue
        if i in seq_to_starts[B]:
            sub_order.append('B')
            i += len(B)
            continue
        if i in seq_to_starts[C]:
            sub_order.append('C')
            i += len(C)
            continue
        break
    if i == len(move_order):
        return sub_order
    else:
        return False

In [ ]:
for A in possible_As():
    for B in possible_Bs(A):
        for C in possible_Cs(A, B):
            possible_sub_order = covers(A, B, C)
            if possible_sub_order:
                sub_order = possible_sub_order
                subroutines = {'A': A, 'B': B, 'C': C}

In [ ]:
main_routine = ','.join(sub_order)
subroutine_strings = '\n'.join([sequence_to_string(subroutines[which]) for which in 'ABC'])
no_output = 'n\n'
whole_program = '\n'.join([main_routine, subroutine_strings, no_output])
ascii_program = [ord(c) for c in whole_program]

In [ ]:
p = intcode.Program(instructions_source='day17.txt', input_queue=ascii_program.copy(), memory_overrides={0: 2})

p.run()

p.output_queue.vals[-1]